<div align='center'>
    <h1>Tugas Besar Dasar Kecerdasan Artifisial</h1>
    <h3>Judul: Penerapan Fuzzy Mamdani dan Sugeno dalam Menentukan Kelayakan Air Minum Berdasarkan Parameter Kualitas Air</h3>
</div>

In [1]:
# Import Library
!pip install scikit-fuzzy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.9 MB/s eta 0:00:00


In [2]:
# Load Dataset
df = pd.read_csv('water_potability.csv')
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [3]:
# Tangani Missing Value
df.isnull().sum()

,0
ph,491
Hardness,0
Solids,0
Chloramines,0
Sulfate,781
Conductivity,0
Organic_carbon,0
Trihalomethanes,162
Turbidity,0
Potability,0


In [4]:
df = df.fillna(df.mean())

In [5]:
# Pilih 3 fitur
df = df[['ph', 'Turbidity', 'Sulfate', 'Potability']]

In [6]:
# Definisikan Variabel Fuzzy Mamdani
ph = ctrl.Antecedent(np.arange(0, 14.1, 0.1), 'ph')
turbidity = ctrl.Antecedent(np.arange(0, 10.1, 0.1), 'turbidity')
sulfate = ctrl.Antecedent(np.arange(0, 500.1, 1), 'sulfate')
potability = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'potability')

In [7]:
# Fungsi Keanggotaan
ph['asam'] = fuzz.trimf(ph.universe, [0, 0, 7])
ph['netral'] = fuzz.trimf(ph.universe, [6, 7, 8])
ph['basa'] = fuzz.trimf(ph.universe, [7, 14, 14])

turbidity['rendah'] = fuzz.trimf(turbidity.universe, [0, 0, 4])
turbidity['sedang'] = fuzz.trimf(turbidity.universe, [2, 5, 8])
turbidity['tinggi'] = fuzz.trimf(turbidity.universe, [6, 10, 10])

sulfate['rendah'] = fuzz.trimf(sulfate.universe, [0, 0, 200])
sulfate['sedang'] = fuzz.trimf(sulfate.universe, [100, 250, 400])
sulfate['tinggi'] = fuzz.trimf(sulfate.universe, [300, 500, 500])

potability['tidak layak'] = fuzz.trimf(potability.universe, [0, 0, 0.5])
potability['layak'] = fuzz.trimf(potability.universe, [0.5, 1, 1])

In [8]:
# Aturan Fuzzy Mamdani
rule1 = ctrl.Rule(ph['netral'] & turbidity['rendah'] & sulfate['rendah'], potability['layak'])
rule2 = ctrl.Rule(ph['asam'] | turbidity['tinggi'] | sulfate['tinggi'], potability['tidak layak'])
rule3 = ctrl.Rule(ph['basa'] & turbidity['sedang'], potability['tidak layak'])
rule4 = ctrl.Rule(ph['netral'] & sulfate['sedang'], potability['layak'])

In [9]:
# Simulasi Mamdani
potability_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4])
potability_sim = ctrl.ControlSystemSimulation(potability_ctrl)

potability_sim.input['ph'] = 7.2
potability_sim.input['turbidity'] = 3.5
potability_sim.input['sulfate'] = 180
potability_sim.compute()

print("Output Mamdani:", potability_sim.output['potability'])

Output Mamdani: 0.7708261007666092


In [10]:
# Implementasi Sugeno
def fuzz_ph(ph_val):
    return {'asam': max(min((7 - ph_val)/7, 1), 0),
            'netral': max(1 - abs(ph_val - 7), 0),
            'basa': max(min((ph_val - 7)/7, 1), 0)}

def fuzz_turbidity(tb):
    return {'rendah': max(min((4 - tb)/4, 1), 0),
            'sedang': max(1 - abs(tb - 5)/3, 0),
            'tinggi': max(min((tb - 6)/4, 1), 0)}

def fuzz_sulfate(sf):
    return {'rendah': max(min((200 - sf)/200, 1), 0),
            'sedang': max(1 - abs(sf - 250)/150, 0),
            'tinggi': max(min((sf - 300)/200, 1), 0)}

In [11]:
def sugeno_inferensi(ph_val, tb, sf):
    f_ph = fuzz_ph(ph_val)
    f_tb = fuzz_turbidity(tb)
    f_sf = fuzz_sulfate(sf)

    rules = []
    rules.append((min(f_ph['netral'], f_tb['rendah'], f_sf['rendah']), 1))
    rules.append((max(f_ph['asam'], f_tb['tinggi'], f_sf['tinggi']), 0))
    rules.append((min(f_ph['basa'], f_tb['sedang']), 0.2))
    rules.append((min(f_ph['netral'], f_sf['sedang']), 0.7))

    numerator = sum([w*z for w, z in rules])
    denominator = sum([w for w, z in rules]) + 1e-6
    return numerator / denominator

In [12]:
# Uji Sugeno
result = sugeno_inferensi(7.2, 3.5, 180)
print("Output Sugeno:", result)

Output Sugeno: 0.7237399137742309
